In [401]:
# pandasのインポート
import pandas as pd

# CSVの読み込み
df1 = pd.read_csv('titanic_kouka2_1.csv', sep = ',')
df2 = pd.read_csv('titanic_kouka2_2.csv', sep = ',')

# 正しく読み込めているかを確認するため、それぞれのデータフレームの内容を表示する
print('===titanic_kouka2_1.csv===')
print(df1.head())

print('===titanic_kouka2_2.csv===')
print(df2.head())

===titanic_kouka2_1.csv===
   乗客ID  生存  乗客クラス     名前      性別
0     1   0      3  name1    male
1     2   1      1  name2  female
2     3   1      3  name3  female
3     4   1      1  name4  female
4     5   0      3  name5    male
===titanic_kouka2_2.csv===
   乗客ID    年齢  兄弟と配偶者  親と子供     チケット番号       運賃  客室番号 出港地
0     1  22.0       1     0  A/5 21171   7.2500   NaN   S
1     2  38.0       1     0   PC 17599  71.2833   C85   C
2     4  35.0       1     0     113803  53.1000  C123   S
3     5  35.0       0     0     373450   8.0500   NaN   S
4     6   NaN       0     0     330877   8.4583   NaN   Q


In [402]:
# データフレームを'乗客ID'列をキーに列方向に連結する
df = df1.merge(right=df2, how='left', on='乗客ID')
print('行列数',df.shape)

行列数 (891, 12)


In [403]:
# 正解データの偏りの有無を確認するため、データの種類ごとの出現回数を表示する
df['生存'].value_counts()

0    549
1    342
Name: 生存, dtype: int64

In [404]:
# 各列に欠損値がどれだけ含まれているかを確認する
# 欠損値がある場合は穴埋めを考えるが、欠損値が多く分析で使えない列かもしれない
df.isnull().sum()

乗客ID        0
生存          0
乗客クラス       0
名前          0
性別          0
年齢        178
兄弟と配偶者      1
親と子供        1
チケット番号      1
運賃          1
客室番号      687
出港地         3
dtype: int64

In [405]:
# 客室番号の列には欠損値が891件中687件もあるため今回は削除する
# 欠損値を削除する
df= df.drop('客室番号', axis = 1)

# 削除されているか確認する
df.isnull().sum()

乗客ID        0
生存          0
乗客クラス       0
名前          0
性別          0
年齢        178
兄弟と配偶者      1
親と子供        1
チケット番号      1
運賃          1
出港地         3
dtype: int64

In [406]:
# 中央値で穴埋めすることとし、モデルの性能が良くなければ別の対処も考えることにする
df_fillna = df.fillna(df.median())

# また中央値で中央値で補完できない行は削除する
df_fillna.dropna(inplace=True)

# 補完されているか確認する
df_fillna.isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


乗客ID      0
生存        0
乗客クラス     0
名前        0
性別        0
年齢        0
兄弟と配偶者    0
親と子供      0
チケット番号    0
運賃        0
出港地       0
dtype: int64

In [407]:
# 文字列データをダミー変数化する
# ダミー変数化の結果が変わらないよう、データ分割前にダミー変数化を実施する
# データフレームを丸ごと渡すことで、文字の列のみダミー変数化し、それ以外は元のままとしたデータフレームを取得できる
df_dummied = pd.get_dummies(df_fillna)

# 正しくダミー変数化されていることを確認するため、内容を表示する
print(df_dummied)

     乗客ID  生存  乗客クラス    年齢  ...  チケット番号_WE/P 5735  出港地_C  出港地_Q  出港地_S
0       1   0      3  22.0  ...                 0      0      0      1
1       2   1      1  38.0  ...                 0      1      0      0
3       4   1      1  35.0  ...                 0      0      0      1
4       5   0      3  35.0  ...                 0      0      0      1
5       6   0      3  28.0  ...                 0      0      1      0
..    ...  ..    ...   ...  ...               ...    ...    ...    ...
886   887   0      2  27.0  ...                 0      0      0      1
887   888   1      1  19.0  ...                 0      0      0      1
888   889   0      3  28.0  ...                 0      0      0      1
889   890   1      1  26.0  ...                 0      1      0      0
890   891   0      3  32.0  ...                 0      0      1      0

[888 rows x 1579 columns]


In [408]:
# データを、特徴量、正解データに分ける
# 特徴量を'乗客クラス', '性別_female', '性別_male', '兄弟と配偶者', '親と子供の列'とする
X = df_dummied[['乗客クラス', '性別_female', '性別_male','兄弟と配偶者', '親と子供']]

# 正解データは'生存'
y = df_dummied[['生存']]

In [409]:
# train_test_splitをインポートする
from sklearn.model_selection import train_test_split

# データを訓練データ＆検証データ、テストデータに分割する
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

In [410]:
# AdaBoostClassifierをインポートする
from sklearn.ensemble import AdaBoostClassifier
# DecisionTreeClassifierをインポートする
from sklearn.tree import DecisionTreeClassifier

# ベースとして使用するモデルを深さ5の決定木(分類)　とする
base_model = DecisionTreeClassifier(random_state = 0, max_depth = 5, class_weight='balanced')

# アダブーストを使用してモデルを作成する
model = AdaBoostClassifier(n_estimators = 500, random_state = 0, base_estimator = base_model)
model.fit(X_train_val, y_train_val)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced',
                                                         max_depth=5,
                                                         random_state=0),
                   n_estimators=500, random_state=0)

In [411]:
# テストデータでの正解率を確認する
print(model.score(X_test, y_test))

0.8202247191011236


In [412]:
# 新しくデータフレームを作成する
predict_data = pd.DataFrame([[3, 'nameA', 'female', 13, 1, 0, '2651', 11.2417, 'C99', 'C'],
                             [2, 'nameb', 'female', 13, 1, 0, '2651', 11.2417, 'C99', 'C'],
                             [2, 'namec', 'male', 13, 1, 0, '2651', 11.2417, 'C99', 'C']], 
                             columns = ['乗客クラス', '名前', '性別', '年齢', '兄弟と配偶者', '親と子供', 'チケット番号', '運賃', '客室番号', '出港地'])
# 作成したデータフレームを表示する
print(predict_data)

   乗客クラス     名前      性別  年齢  兄弟と配偶者  親と子供 チケット番号       運賃 客室番号 出港地
0      3  nameA  female  13       1     0   2651  11.2417  C99   C
1      2  nameb  female  13       1     0   2651  11.2417  C99   C
2      2  namec    male  13       1     0   2651  11.2417  C99   C


In [413]:
# 性別列をダミー変数化する
predict_data = pd.get_dummies(predict_data)

# ダミー変数化した結果を表示する
print(predict_data)

   乗客クラス  年齢  兄弟と配偶者  親と子供  ...  性別_male  チケット番号_2651  客室番号_C99  出港地_C
0      3  13       1     0  ...        0            1         1      1
1      2  13       1     0  ...        0            1         1      1
2      2  13       1     0  ...        1            1         1      1

[3 rows x 13 columns]


In [414]:
# 必要な列のみ取得する
predict_data = predict_data.loc[:, ['乗客クラス', '性別_female', '性別_male', '兄弟と配偶者', '親と子供']]

In [415]:
# 以下のデータをもとにモデルで予測した結果を表示する
model.predict(predict_data)

array([1, 1, 0])